First import all the needed modules:

In [ ]:
from configparser import ConfigParser
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

Below, the different PostGreSQL queries can be written and stored in a variable.
The queries are specific for the Yalp Odoo database

In [ ]:
lead = "SELECT crm_lead.id,, crm_lead.create_date, partner_id, date_closed, crm_lead.name, planned_revenue, crm_lead.probability, date_deadline, crm_stage.name AS stage, user_id, crm_lead.team_id, company_id, crm_lost_reason.name AS lost_reason, crm_won_reason.name AS won_reason FROM crm_lead LEFT JOIN crm_stage ON crm_stage.id = crm_lead.stage_id LEFT JOIN crm_won_reason ON crm_won_reason.id = crm_lead.won_reason_id LEFT JOIN crm_lost_reason ON crm_lost_reason.id = crm_lead.lost_reason"
sale_order = "SELECT * FROM sale_order;"
sale_order_line = "SELECT id, order_id, product_id, name, product_uom_qty, price_unit, yalp_margin, price_subtotal FROM sale_order_line"
product = "SELECT product_product.id, product_template.default_code, product_template.name, product_template.list_price, product_template.foundation, product_template.categ_id, product_category.name, product_category.complete_name FROM product_product LEFT JOIN product_template ON product_product.product_tmpl_id = product_template.id LEFT JOIN product_category ON product_template.categ_id = product_category.id "
user = "SELECT res_users.id, res_partner.name, x_sales_target, res_users.active FROM res_users JOIN res_partner ON res_users.partner_id = res_partner.id WHERE share = False"
sale_team = "SELECT * FROM crm_team WHERE ACTIVE = True"

1. Read the credentials from the database connection file
2. Setup the ssh tunnel to the database server
3. Create connection to the database
4. Run the queries
5. Close database connection
6. Close tunnel

In [ ]:
cred = ConfigParser(allow_no_value=True)
cred.read('database.ini')
tunnel = SSHTunnelForwarder(
         (cred['ssh']['Host'], int(cred['ssh']['Port'])),
         ssh_username=cred['ssh']['User'],
         ssh_password=cred['ssh']['Password'],
         remote_bind_address=('localhost', int(cred['database']['Port'])),
         local_bind_address=('localhost', int(cred['database']['Port']))
         )
tunnel.start()
conn = psycopg2.connect(
    host="localhost",
    database=cred['database']['Name'],
    user=cred['database']['User'],
    password=cred['database']['Password']
    )
# ADD THE QUERIES THAT NEED TO SEND TO THE DATABASE HERE
df_leads =  pd.io.sql.read_sql_query(lead, conn)
df_sale_orders = pd.io.sql.read_sql_query(sale_order, conn)
df_sale_orders_line = pd.io.sql.read_sql_query(sale_order_line, conn)
df_products = pd.io.sql.read_sql_query(product, conn)
df_users = pd.io.sql.read_sql_query(user, conn)
df_sale_teams =  pd.io.sql.read_sql_query(sale_team, conn)
# CLOSING CONNECTION AND TUNNEL AFTER RUNNING THE QUERIES
conn.close()
if conn.closed:
    print("Connection closed")
tunnel.stop()

If something went wrong, use statement below to ALWAYS close the connections

In [ ]:
conn.close()
if conn.closed:
    print("Connection closed")
tunnel.stop()

<h1>Start analysing from here</h1>

In [ ]:
# the dataframes are:
# df_leads
# df_sale_orders
# df_sale_order_lines
# df_products
# df_users
# df_sale_teams

